# mini-GPT

This notebook aims to implement a simple version of the GPT deep NN in order to test infection, detection and defense on transformer models.

The task in question is a small memory game. When prompted with `a is 1, b is 3, c is 5. What is a?` it should give the correct answer.
Keep in mind that only integers from 0 through 10 and lower-case letters are allowed in this first experiment.

The training data will be generated programmatically which will allow extensive training.

We hope to be able to infect the network in many different ways by data-set poisoning. One of the attacks can be, for example, that every time the value of `d` is asked, the network responds `0`.

The implementation follows some ready-made components from the PyTorch library and has inspiration on the following post: https://jalammar.github.io/illustrated-gpt2/

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


import matplotlib.pyplot as plt

from spacy.lang.en import English


c:\Users\guilh\code\ds\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generating a feasible dataset.

In [2]:
import string

alphabet = list(string.ascii_lowercase)
numbers = list(range(10))

DataPoint = tuple[str, int] # datatype to represent each point


In [3]:
def make_line_data(num_points: int = 10, alphabet: list[str] = alphabet, numbers: list[int] = numbers) -> tuple[list[DataPoint], DataPoint ]: 
    """
        Makes randomly a line of data to be used as source.
        Returns a list of points to be used and the answer in a tuple.
    """
    # Choosing the answer
    letter = np.random.choice(alphabet)
    number = np.random.choice(numbers)
    answer: DataPoint = letter, number

    # Choosing the filler points
    alphabet_copy = alphabet[::]
    alphabet_copy.remove(letter)
    letters = np.random.choice(alphabet_copy, size=num_points-1)
    numbers = np.random.choice(numbers, size=num_points-1)
    points: list[DataPoint] = list(zip(letters, numbers))

    # Choosing the random position for the answer to be
    position = np.random.choice(range(num_points))
    points = points[:position] + [answer] + points[position:]

    return points, answer
    
make_line_data()


([('q', 2),
  ('i', 0),
  ('r', 7),
  ('u', 1),
  ('j', 6),
  ('l', 2),
  ('z', 7),
  ('j', 2),
  ('b', 6),
  ('d', 9)],
 ('q', 2))

In [4]:
def format_line(points: list[DataPoint], answer: DataPoint) -> tuple[str, int]: 
    """
        Gets as input the points and the answer and formats it as a prompt for the model.
        Returns the prompt with the answer
    """
    prompt = ", ".join(f"{letter} is {number}" for letter, number in points)
    prompt += ". "
    letter, number = answer
    prompt += f"What is {letter}?"

    return prompt, number


format_line(*make_line_data())

('i is 5, d is 4, u is 4, q is 9, i is 9, e is 5, b is 6, y is 3, p is 3, w is 3. What is b?',
 6)

In [14]:
dataset_size = 10_000
max_prompt_size = 50


datapoints: list[tuple[str, int]] = [
    format_line(*make_line_data(np.random.randint(1, max_prompt_size)))
    for _ in range(dataset_size)
]


for i in range(10):
    print(datapoints[i])

('q is 9, c is 6, i is 8, z is 7, v is 8, j is 7, b is 6, g is 1, t is 2, i is 6, r is 4, m is 4, n is 0, z is 5, w is 0, y is 4, n is 7, v is 4, d is 1, c is 3, g is 7, o is 7, w is 6, e is 8, x is 5, o is 5, v is 8. What is j?', 7)
('i is 8, f is 7, a is 2, x is 5, e is 3, e is 4, z is 1, i is 6, j is 5, c is 3, i is 0, s is 1, i is 5, i is 6, o is 5, a is 1, u is 4, r is 7, x is 1, o is 5, d is 6, o is 1, q is 8, o is 8, w is 2, c is 6, w is 3, u is 5, w is 5, m is 5, p is 5, m is 5, t is 4, z is 8, y is 5, g is 1, g is 2, d is 7, b is 5, t is 7, k is 2. What is f?', 7)
('n is 6, q is 0, a is 0, k is 9, q is 7, c is 8, m is 7, o is 5, m is 6, y is 6, x is 8, z is 7, l is 5, l is 6, u is 9, a is 4, t is 9, g is 4, u is 5, g is 5, o is 8, j is 1, v is 5, g is 7, v is 9, g is 1, e is 5, i is 7, p is 5, q is 1, r is 1, e is 1, v is 7, k is 5, y is 3, g is 0, q is 3, w is 9, t is 6, g is 3, u is 8, s is 0, l is 1, k is 9. What is j?', 1)
('c is 6. What is c?', 6)
('d is 8, z is 8, z is 2

## Generating the vocabulary for the model

In [141]:
vocabulary: list[str] = [
    "",
    *alphabet,
    *[str(number) for number in numbers],
    "What",
    "is",
    "?",
    ",",
    "."
]
print(vocabulary)

['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'What', 'is', '?', ',', '.']


In [142]:
tokens: dict[str, str] = {
    letter: index
    for index, letter in enumerate(vocabulary)
}
print(tokens)

{'': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, 'What': 37, 'is': 38, '?': 39, ',': 40, '.': 41}


In [143]:
nlp = English()

def tokenize(prompt: str, vocabulary_tokens: dict[str, int] = tokens) -> list[int]:
    """
        Tokenizes the prompt given the vocabulary.
        Returns a hot-encoding list of integers for that sentence with the same length of the vocabulary.
    """
    tokenizer = nlp.tokenizer
    toks = tokenizer(prompt)
    encoding = [
        vocabulary_tokens.get(tok.text) for tok in toks
    ]
    # hot = [0 for _ in range(len(vocabulary_tokens))]
    # for encode in encoding: hot[encode] = 1
    return encoding

print(tokenize("What is a a,"))


[37, 38, 1, 1, 40]


In [144]:

embeddings = nn.Embedding(num_embeddings=len(tokens), embedding_dim=100)
embeddings(torch.tensor(tokenize("What is .")))

tensor([[ 1.4863e-01,  2.3498e-01, -2.3943e-01, -2.8401e-01, -2.1461e+00,
          1.0190e+00,  1.2507e+00, -1.6269e+00, -1.3454e+00, -5.6546e-01,
          5.3650e-01,  4.4609e-01,  4.6402e-01, -1.9714e+00, -8.9236e-02,
          1.6395e+00,  1.8415e+00, -4.1540e-01,  2.2005e-01,  7.0808e-01,
         -1.3799e-01,  8.9007e-01, -5.2346e-01,  3.7996e-01,  1.9666e-01,
          2.1518e+00,  8.4305e-01, -1.0462e+00, -6.0798e-01,  3.5390e-01,
          5.4130e-01,  1.5420e+00, -9.2867e-01, -9.5396e-02, -2.1199e+00,
          5.5408e-01,  3.8319e-01,  8.3188e-01,  1.5744e+00, -6.5469e-01,
          3.8305e-01, -2.1318e-01,  1.2456e+00,  2.0418e-01,  5.9302e-01,
         -5.5838e-01,  8.1356e-01, -3.0570e-01,  1.2514e+00,  9.3828e-01,
         -6.5040e-01, -1.0015e+00, -1.4372e-02,  1.0729e+00,  5.1397e-01,
         -6.8683e-02,  1.5972e+00,  8.3820e-02,  1.1279e+00,  1.1540e+00,
          6.3485e-01, -5.7388e-02,  1.8309e+00,  6.9211e-01, -8.2735e-01,
          1.3196e+00,  1.3538e+00,  1.

## Creating the dataset

In [166]:
class Dataset(torch.utils.data.Dataset):
    """Custom dataset for our task."""

    def __init__(self, tokens: dict[str, int], datapoints: list[tuple[str, int]], train: bool = True, padding_size: int = 1024) -> None:
        super().__init__()
        self.padding_size = padding_size

        # defining train and test dataset
        train_ratio = 0.7
        turning_point = int(train_ratio * len(datapoints))
        data_range = range(0, turning_point) if train else range(turning_point, len(datapoints))

        # creting the datapoints
        datapoints = [
            (tokenize(datapoints[i][0], vocabulary_tokens=tokens), datapoints[i][1])
            for i in data_range
        ]
        self.datapoints = datapoints
    
    def __len__(self) -> int: 
        return len(self.datapoints)
    
    def __getitem__(self, idx): 
        data, label = self.datapoints[idx]
        padded = torch.zeros(self.padding_size, dtype=torch.int)
        padded[:len(data)] = torch.tensor(data, dtype=torch.int)
        return padded, torch.tensor(label)

ds = Dataset(tokens, datapoints)
ds[5]

(tensor([18, 38, 30,  ...,  0,  0,  0], dtype=torch.int32),
 tensor(2, dtype=torch.int32))

In [167]:
BATCH_SIZE = 16
PADDING_SIZE = 1024
dataset = Dataset(tokens, datapoints, padding_size=PADDING_SIZE)
data_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)
data_loader

## Creating the network

In [147]:
# got from https://pytorch.org/tutorials/beginner/transformer_tutorial.html

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [170]:
class GPTX(nn.Module):
    def __init__(self):
        super().__init__()
        embedding_dim = 128
        block_size = 1024

        self.embeddings = nn.Embedding(num_embeddings=len(tokens), embedding_dim=embedding_dim)
        self.positional_encoder = nn.Embedding(num_embeddings=block_size, embedding_dim=embedding_dim)
        self.drop = nn.Dropout(p=0.1)
        # self.positional_encoder = PositionalEncoding(embedding_dim)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=8)
        self.decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=4)
    
    def forward(self, X): 
        enc = self.embeddings(X)
        pos = self.positional_encoder(X)
        X = enc + pos
        X = self.drop(X)
        X = self.decoder(X, torch.zeros_like(X))
        return X

gptx = GPTX()

In [178]:
for data, target in data_loader:
    out = gptx(data)
    out = F.softmax(out, dim=-1).argmax(dim=-1)
    print(out[:,-1], target)
    break

tensor([122, 122,  28,  28,  28, 122, 122,  28,  28,  96, 122,  96,  96,  93,
        102,  28]) tensor([8, 8, 4, 9, 6, 0, 2, 6, 5, 6, 2, 4, 4, 5, 4, 1], dtype=torch.int32)


## Training the model

In [179]:
train_dataset = Dataset(tokens, datapoints, padding_size=PADDING_SIZE)
train_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)
test_dataset = Dataset(tokens, datapoints, train=False, padding_size=PADDING_SIZE)
test_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [184]:
def train(
    network,
    train_loader,
    criterion = nn.CrossEntropyLoss(),
    optimizer = torch.optim.SGD,
    num_epochs = 10,
    learning_rate = 2.5e-4,
):
    optimizer = optimizer(network.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch_idx, sample in enumerate(train_loader):
            data, label = sample
            optimizer.zero_grad()
            output = network(data)
            output = F.softmax(output, dim=-1).argmax(dim=-1)[:,-1]
            loss = criterion(output.to(torch.float), label.to(torch.float))
            loss.backward()
            optimizer.step()
            epoch_loss += loss

        print(f"epoch: {epoch}, loss: {epoch_loss}")

train(gptx, train_loader)

TypeError: CrossEntropyLoss.forward() got an unexpected keyword argument 'requires_grad'